<a href="https://colab.research.google.com/github/prateekkosta/Big-Mart-sales-analysis-/blob/main/BigMart_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Objective**
Predict the sales of each product by understanding product properties and ourlet sales by implementing some Machine learning models.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mode
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder

In [ ]:
df= pd.read_csv('/content/drive/MyDrive/Train big mart.csv')

In [ ]:
df.head(10)

In [ ]:
# Checking statical features of Dataset
df.describe()

In [ ]:
#Checking for number of Rows and Columns

df.shape

In [ ]:
#Checking for type of data in each column

df.info()

As we can see from dataset info that Item_Weight and Outlet_Size has some missing values which we have to impute.

In [ ]:
#Checking for Unique values
df.nunique()

As we can see that Dataset has 1559 type of products, 16 type of items, 3 type of outlet size, 3 type of Outlet Location and 4 type of outlet type.

In [ ]:
# Checking for Null values
df.isnull().sum()

Columns Item_Weight contains 1463 null values and Outlet_Size contains 2410 Null values.

In [ ]:
# Now we will look for Object Data type columns i.e. columns which has categorical type of data.

object_column= []
for i in df:
  if df[i].dtype== 'object':
    object_column.append(i)
object_column

Since Item_Identifier and Outlet_Identifier are just unique ids provided to products and stores, so we can remove them for our analysis.

In [ ]:
object_column.remove('Outlet_Identifier')
object_column.remove('Item_Identifier')
object_column

Now checking that which type of data are present in these columns and how many type of data is present.

In [ ]:
#Checking for data type and type of Data

for i in object_column:
  print(i)
  print(df[i].value_counts())
  print()


## Missing Value Imputation

Now I will replace the mean value of different products acccording to their type.

In [ ]:
item_weight_mean= df.groupby('Item_Identifier').agg({'Item_Weight': np.mean})
item_weight_mean

In [ ]:
# finding boolean values of missing data.

missing_item_weight= df['Item_Weight'].isnull()
missing_item_weight

Now I will look at location where boolean is true and check for product type in that locations and than replace missing values with mean of same products types.

In [ ]:
for i, item in enumerate(df['Item_Identifier']):
  if missing_item_weight[i]:
    if item in item_weight_mean:
      df['Item_Weight'][i]= item_weight_mean.loc['item']['item_weight']
    else:
      df['Item_Weight'][i]= np.mean(df['Item_Weight'])

In [ ]:
df['Item_Weight'].isnull().sum()

Now finding outlet type with their respective mode values.

In [ ]:
outlet_size_mode= df.pivot_table(values= 'Outlet_Size', columns= 'Outlet_Type', aggfunc=( lambda x: x.mode([0])) )

In [ ]:
outlet_size_mode

In [ ]:
missing_outlet= df['Outlet_Size'].isnull()
missing_outlet

In [ ]:
#Replaccing values in column

df.loc[missing_outlet, 'Outlet_Size']= df.loc[missing_outlet, 'Outlet_Type'].apply(lambda x: outlet_size_mode[x])

In [ ]:
df['Outlet_Size'].isnull().sum()

**From the describe function we have seen that item visibility has 0 values which makes no practical sense. So we will replace 0 Value with mean of Item_visibility.**

In [ ]:
(df['Item_Visibility']==0).sum()

In [ ]:
df.loc[:,'Item_Visibility'].replace([0], [df['Item_Visibility'].mean()], inplace= True)

In [ ]:
(df['Item_Visibility']==0).sum()

As we can see from the Data that Item_fat_content column has similar type of values with multiple names like Low Fat is also written as LF, low fat and Regular is written as Reg. So we will make it as same type.

In [ ]:
df['Item_Fat_Content']= df['Item_Fat_Content'].replace({'LF':'Low Fat', 'low fat': 'Low Fat', 'reg':'Regular'})
df['Item_Fat_Content'].value_counts()

# **Feature Engineering**

Now we will extract first two words from Item Identifiers to make a new column New Item Type which will define weather it is food, drinking or Non Consumable item.

In [ ]:
df['New_Item_Type']= df['Item_Identifier'].apply(lambda x: x[:2])
df['New_Item_Type']

In [ ]:
df['New_Item_Type']= df['New_Item_Type'].map(({'FD': 'Food', 'DR': 'Drinking', 'NC': 'Non-Consumable'}))

In [ ]:
df.New_Item_Type.value_counts()

No we will check the New Items which are Non-Consumable and if they contain fat make it non edible item.

In [ ]:
df.loc[df['New_Item_Type']== 'Non-Consumable', 'Item_Fat_Content']= 'Non-Edible'
df['Item_Fat_Content'].value_counts()

In [ ]:
perishable_item= ["Breads", "Breakfast", "Dairy", "Snack Foods","Fruits and Vegetables", "Meat", "Seafood", "Starchy Foods"]
non_perishable_item= ["Baking Goods", "Canned", "Frozen Foods", "Hard Drinks", "Health and Hygiene","Household", "Soft Drinks"]


In [ ]:
def New_Item_Type(item):
  if item in perishable_item:
    return 'Perishable'
  elif item in non_perishable_item:
    return 'Non-Perishable'
  else:
    return 'Not-Known'

In [ ]:
df['Shelf_Life']= df['Item_Type'].apply(New_Item_Type)

In [ ]:
df['Shelf_Life'].value_counts()

In [ ]:
df['MRP_per_unit_weight']= df['Item_MRP']/ df['Item_Weight']

In [ ]:
df['Outlet_years']= 2013 - df['Outlet_Establishment_Year']

In [ ]:
df.head()

# **Exploratory Data Analysis**

Visualization of Numerical features.

In [ ]:
plt.figure(figsize= (8,6))
sns.distplot(df['Item_Weight'],bins= 20)

In [ ]:
plt.figure(figsize= (8,6))
sns.distplot(df['Item_Visibility'], bins=20)

In [ ]:
df['Item_Visibility']= np.log(df['Item_Visibility'])

In [ ]:
sns.distplot(df['Item_Visibility'], bins= 20)


In [ ]:
sns.distplot(df['Item_MRP'], bins= 20)

In [ ]:
sns.distplot(df['Item_Outlet_Sales'], bins= 20)

Here item outlet sale is Right skewed and we have to make it normally distributed, so applying log.

In [ ]:
df['Item_Outlet_Sales']= np.log(1+ df['Item_Outlet_Sales'])

In [ ]:
sns.distplot(df['Item_Outlet_Sales'], bins= 20)

**Visualization of Categorical features**

In [ ]:
df['Item_Type']=df['Item_Type'].astype(str)

In [ ]:
plt.figure(figsize= (8,6))
Item_Type_Vis= sns.countplot(x= 'Item_Type', data= df)
Item_Type_Vis.set_xticklabels(Item_Type_Vis.get_xticklabels(), rotation= 80)
None

In [ ]:
Outlet_Type_Vis= sns.countplot(x= 'Outlet_Type', data= df)
Outlet_Type_Vis.set_xticklabels(Outlet_Type_Vis.get_xticklabels(), rotation= 40)
None

In [ ]:
Outlet_Size_Vis= sns.countplot(x= 'Outlet_Size', data= df)
Outlet_Size_Vis.set_xticklabels(Outlet_Size_Vis.get_xticklabels())
None

In [ ]:
Item_Fat_Content_Vis= sns.countplot(x= "Item_Fat_Content", data= df)
Item_Fat_Content_Vis.set_xticklabels(Item_Fat_Content_Vis.get_xticklabels())
None

In [ ]:
Outlet_Location_type_Vis= sns.countplot(x='Outlet_Location_Type', data= df)
Outlet_Location_type_Vis.set_xticklabels(Outlet_Location_type_Vis.get_xticklabels())
None

# **Bivariate Analysis**

In [ ]:
#Now We will check Average sales Outlet location Type wise

Outlet_location_Sales= sns.barplot(x= 'Outlet_Location_Type', y= np.exp(df['Item_Outlet_Sales']), data= df)
Outlet_location_Sales.set_xticklabels(Outlet_location_Sales.get_xticklabels())
plt.title('Outlet Location Type Vs Item Outlet Sales', fontsize= 16)
None

In [ ]:
# Checking for Average sales Outlet Type wise

Outlet_type_sales= sns.barplot(x= 'Outlet_Type', y= np.exp(df['Item_Outlet_Sales']), data= df)
Outlet_type_sales.set_xticklabels(Outlet_type_sales.get_xticklabels(), rotation= 80)
plt.title('Outlet Type vs Item Outlet Sales')
None

In [ ]:
# Checking for Avg sales Outlet size wise.

Outlet_Size_sales= sns.barplot(x= 'Outlet_Size', y= np.exp(df['Item_Outlet_Sales']), data= df)
Outlet_Size_sales.set_xticklabels(Outlet_Size_sales.get_xticklabels(), rotation= 80)
plt.title('Outlet Size vs Item Outlet Sales')
None

In [ ]:
plt.figure(figsize= (15,8))
Item_type_sales= sns.barplot(x='Item_Type', y= np.exp(df['Item_Outlet_Sales']), data= df)
Item_type_sales.set_xticklabels(Item_type_sales.get_xticklabels(), rotation= 80)
plt.title('Item Type Vs Item Outlet Sales')
None

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize= (8,6))
sns.heatmap(df.corr(), annot= True)
None

There is no significant Correlation observed except Item MRP vs Item Outlet Sales because if Item MRP increase sales is also increased.

## **Now we will check outlet total sales through Pivot table with diffrent features.**

In [ ]:
import pandas as pd
pd.options.display.float_format= '{:.2f}'.format
df.pivot_table(values= 'Item_Outlet_Sales', index=['Outlet_Location_Type', 'Outlet_Type', 'Outlet_Size'], aggfunc= np.sum)

In [ ]:
df.pivot_table(values=  'Item_Outlet_Sales', index=[ 'Item_Fat_Content', 'New_Item_Type', 'Item_Type' ], aggfunc= np.sum)

In [ ]:
df.pivot_table(values= "Item_Outlet_Sales", index= ['Outlet_Size', 'Outlet_Identifier'], aggfunc= np.sum)

# **Data Preprocessing**

**Label Encoding**

In [ ]:
#Doing label incoding for variables which has internal dependency.

from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()

In [ ]:
df['Outlet']= le.fit_transform(df['Outlet_Identifier'])

### **One Hot Encoding**

In [ ]:
#One Hot Encoding for variables which has no internal dependency.

df= pd.get_dummies(df, columns=['Outlet_Type','Item_Fat_Content','New_Item_Type','Outlet_Size','Outlet_Location_Type'])

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.dtypes

### **Spliting the Data into Train and Test**

In [ ]:
train= df.drop(['Item_Outlet_Sales', 'Item_Identifier', 'Item_Type', 'Outlet_Identifier', 'Shelf_Life'], axis= 1)
test= df['Item_Outlet_Sales']

In [ ]:
print(train.shape)
print(test.shape)

## *Statical Analysis*

In [ ]:
import statsmodels.api as sm

In [ ]:
x= train
y=test
x= sm.add_constant(x)
result= sm.OLS(y,x).fit()
print(result.summary())

# **Observations**


1.   P value for F static is < 0.05 so we can say that our model is significant(at least for one independent variable the the regression coef. is not equal to zero, rejecting the Null Hypothesis. In this the Null Hypothesis is 'All the regression coef. are equal to zero.
2.   R-Squared value indicate that 72.1 percentage of variance is explained by our model. Adjusted R-square is less than R-square which indicate that model has some insignificant attributes.
3.  Checked for P Values, If P-value is < 0.05 we will say that attribut is contributing to model that means rejecting the Null Hypothesis and if P- value is >0.05 it means the atribute is insignificant that means accepting the Null Hypothesis.
4.  Looking at P-values we can say that attributes ['Item_Weight','Item_Visibility','MRP_per_unit_weight','Item_Fat_Content_Low Fat', 'Item_Fat_Content_Non-Edible','New_Item_Type_Drinking','New_Item_Type_Food','Item_Fat_Content_Regular', 'New_Item_Type_Non-Consumable','Outlet_Location_Type_Tier 2'] has no cobtribution in dependent variables.
5.  We can say that these attributes are not affecting sales.
6.  Prob(Omnibus)- One of the assumption of OLS method is that errors are normally distributed and Omnibus test is performed to check normal distribution. Here the null hypothesis is that the errors are normally dirtibuted. Prob(Omnibus) is supposed to be closed to 1 in order to satisfy OLS method but in this case Prob(Omnibus) is close to 0.00 which means OLS method is not satisfied, errors are not normally distributed.
7.  Durbin-Watson- The value of this is 2.008 which is close to 2 this means this data has no sutocorelation.
8.  Prob(Jarque-Bera)- It is in line with Omnibus test. It is also performed for destribution analysis of regression errors. It is supposed to agree with the Omnibus test and large value of JB Test indicate that errors are not normally distributed.

In [ ]:
train_1= train.drop(['Item_Weight','Item_Visibility','MRP_per_unit_weight','Item_Fat_Content_Low Fat',
                     'Item_Fat_Content_Non-Edible','New_Item_Type_Drinking',
                     'New_Item_Type_Food','Item_Fat_Content_Regular', 'New_Item_Type_Non-Consumable','Outlet_Location_Type_Tier 2'], axis=1)

In [ ]:
x= train_1
y= test

x= sm.add_constant(x)
result= sm.OLS(y,x).fit()
print(result.summary())

## **Observation**


1.   F- Static is increased by a significant amount, so model is significant.
2.   R-Squares is decreased it means that dropped attributes are insignificant for model.
3.  The p values of all the attributes are <0.05 that means all the selected attributes are significant to dependent variable.



## **Splitting Data into Train and Test**

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [ ]:
x_train, x_test, y_train, y_test= train_test_split(train_1, test, test_size= .25, shuffle= True, random_state= 100)

## **Modeling**

In [ ]:
def model_details(model, algo):
  y_pred= model.predict(x_test)
  rmse= np.sqrt(mean_squared_error(y_test, y_pred))
  mae= mean_absolute_error(y_test, y_pred)
  accu= round(model.score(x_test, y_test)*100,2)
  cvs= cross_val_score(model, x_test, y_test, cv= 5)
  mean= round(cvs.mean()*100,2)
  std= round(cvs.std()*2,2)
  print("Model Report")
  print('Acuuracy of {}: {}%'.format(algo, accu),)
  print('RMSE value: ', round(rmse,2))
  print('Cross Validation Score : Mean - {}| Std - {}'.format(mean, std))
  print('MAE value: ', round(mae, 2))


### **Base Model**

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
base_model= np.exp(test.mean())
base_model

In [ ]:
base_model= [base_model]* len(test)

In [ ]:
base_rmse=  np.sqrt(mean_squared_error(np.exp(test), base_model))
base_mae= mean_absolute_error(np.exp(test), base_model)

In [ ]:
print(base_rmse)
print(base_mae)

## **Linear Regression**

In [ ]:
from sklearn.linear_model import LinearRegression
LR= LinearRegression()

In [ ]:
LR.fit(x_train, y_train)
print(LR.score(x_test, y_test))

In [ ]:
y_pred= LR.predict(x_test)
y_pred= np.exp(y_pred)
y_test_lr= np.exp(y_test)

In [ ]:
rmse= np.sqrt(mean_squared_error(y_test_lr, y_pred))
mae= mean_absolute_error(y_test_lr, y_pred)

In [ ]:
print("The RMSE for linear Regression is : ", rmse)
print("The MAE for Liner Regression is : ", mae)

In [ ]:
model_details(LR, 'LinearRegression')

### **Random Forest Regression**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
RF= RandomForestRegressor(n_estimators= 600, min_samples_leaf= 100, min_samples_split= 8, max_depth= 8 )
RF.fit(x_train, y_train)

In [ ]:
RF.score(x_test, y_test)

In [ ]:
y_pred_rf= RF.predict(x_test)
y_pred_rf= np.exp(y_pred_rf)

In [ ]:
rmse=np.sqrt(mean_squared_error(y_test_lr,y_pred_rf))
mae= mean_absolute_error(y_test_lr, y_pred_rf)

In [ ]:
print("The RMSE for Random Forest is : ", rmse)
print("The MAE for Random Forest is : ", mae)

In [ ]:
model_details(RF, "RandomForestRegressor")

### **XG Boost**

In [ ]:
!pip install xgboost

In [ ]:
from xgboost import XGBRegressor
XG= XGBRegressor(learning_rate= 0.05, subsample= 1, max_depth= 2, n_estimator= 400)

In [ ]:
XG.fit(x_train, y_train)
XG.score(x_test, y_test)

In [ ]:
y_pred_xg= XG.predict(x_test)
y_pred_xg= np.exp(y_pred_xg)

In [ ]:
rmse= np.sqrt(mean_squared_error(y_test_lr, y_pred_xg))
mae= mean_absolute_error(y_test_lr, y_pred_xg)

In [ ]:
print("The RMSE for XGB Regressor is : ", rmse)
print("The MAE for XGB Regressor is : ", mae)

In [ ]:
model_details(XG, "XGBRegressor")

### **Comapring Actual sales vs Predicted Sales**
As XGB has given the best performance, so i will check with XGB regressor.

In [ ]:
y_preds_random= XG.predict(x_test)

In [ ]:
pred_xgb= pd.DataFrame(y_preds_random, columns= ['predicted_xgb'])
true_values= list(y_test.values)
pred_xgb['true_value']= true_values

In [ ]:
pred_xgb

In [ ]:
a= np.exp(pred_xgb)
a.describe()

In [ ]:
comp= pred_xgb
comp= comp.iloc[1000:1050]
true_value= comp['true_value']
predicted_values= comp['predicted_xgb']

In [ ]:
plt.plot(true_value)
plt.plot(predicted_values)
plt.ylabel('Item Outlet Sale')
plt.legend(['Actual', "Predicted"])
plt.title('Item Outlet Sales----> Actual vs Predicted', fontsize= 16)
plt.show

## **Observation**
The RMSE and R squared value is comparatively better for XGB regressor and their R2 score is 72% so we will consider this models according to business requirement , Later on we can try to hyper tune the models and check for the optimum results.

In [ ]:
from IPython.display import clear_output
clear_output()